In [19]:
pip install scipy==1.12

DEPRECATION: Loading egg at /Users/omisesan/anaconda3/lib/python3.11/site-packages/endtoendmlgit-0.0.1-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/omisesan/anaconda3/lib/python3.11/site-packages/fonttools-4.55.2-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install matplotlib seaborn


DEPRECATION: Loading egg at /Users/omisesan/anaconda3/lib/python3.11/site-packages/endtoendmlgit-0.0.1-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/omisesan/anaconda3/lib/python3.11/site-packages/fonttools-4.55.2-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [60]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler

In [22]:
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
]

In [23]:
print("Loading 20 newsgroups dataset for categories:")
print(categories)

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc']


In [24]:
data = fetch_20newsgroups(subset='train', categories=categories)
print(f"{len(data.filenames)} documents")
print(f"{len(data.target_names)} categories")
print()

857 documents
2 categories



In [25]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [42]:
models = {
    'MultinomialNB': MultinomialNB(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVM': SVC(),
    'DecisionTree': DecisionTreeClassifier()
}

In [49]:
grid_params = {
    'MultinomialNB': {
    
        'clf__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]
        
    },
    'LogisticRegression': {
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__max_iter': (20, 100),
        'clf__penalty': ('l2', 'elasticnet'),
    },
    'SVM': {
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__ngram_range': ((1, 1), (1, 2)),
    },
    'DecisionTree': {
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__max_depth': [10, 50, None],
    }
}



In [61]:
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.w2v_model = Word2Vec(sentences=[text.split() for text in X], vector_size=100, window=5, min_count=2, workers=4)
        return self
    
    def transform(self, X):
        transformed = np.array([np.mean([self.w2v_model.wv[word] for word in text.split() if word in self.w2v_model.wv], axis=0) if text.split() else np.zeros(100) for text in X])
        return MinMaxScaler().fit_transform(transformed)

class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.d2v_model = Doc2Vec([TaggedDocument(words=text.split(), tags=[i]) for i, text in enumerate(X)], vector_size=100, window=5, min_count=2, workers=4, epochs=20)
        return self
    
    def transform(self, X):
        transformed = np.array([self.d2v_model.infer_vector(text.split()) for text in X])
        return MinMaxScaler().fit_transform(transformed)

In [62]:
best_pipelines = {}
for model_name, model in models.items():
    pipeline = Pipeline([
            ('vect', CountVectorizer()),
            ('clf', model),
        ])
    
    

In [63]:
params = grid_params[model_name]
grid_search = GridSearchCV(pipeline, params, cv=5, n_jobs=1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf', DecisionTreeClassifier())]),
             n_jobs=1,
             param_grid={'clf__max_depth': [10, 50, None],
                         'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=1)

In [64]:
best_pipelines[model_name] = grid_search.best_estimator_
print(f"Best score for {model_name} with CountVectorizer: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(params.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score for DecisionTree with CountVectorizer: 0.874
Best parameters set:
	clf__max_depth: None
	vect__max_df: 1.0
	vect__ngram_range: (1, 1)


In [65]:
results = [['Feature Extractor', 'Model', 'Accuracy']]


In [66]:
for model_name, best_pipeline in best_pipelines.items():
    y_pred = best_pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results.append(['CountVectorizer', model_name, acc])

In [67]:
for feature_name, transformer in [('CountVectorizer', CountVectorizer()), ('Word2Vec', Word2VecTransformer()), ('Doc2Vec', Doc2VecTransformer())]:
    for model_name, model in models.items():
        pipeline = Pipeline([
            (feature_name, transformer),
            ('clf', model),
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        results.append([feature_name, model_name, acc])

In [68]:
results_df = pd.DataFrame(results[1:], columns=results[0])
results_df.to_csv("Abdul-rasaq_Task0_Text_Classification.txt", sep='\t', index=False)